# Analysis of lysine deserts in selected Orthologous Groups (OGs) from the eggNOG5 database

# 1. Import libraries and read data

In [ ]:
import pandas as pd
pd.options.display.max_columns = None
import numpy as np
import math
import statistics
import scipy.stats as stats

import pickle
import gzip
import os
import re

import seaborn as sns
import matplotlib.pyplot as plt
import collections
from IPython.display import Markdown, display

from Bio import AlignIO
from ete3 import NCBITaxa, Tree, TreeStyle, AttrFace, TextFace, add_face_to_node

ncbi = NCBITaxa()

######### Read preprocessed data obtained by running Download_data.ipynb #########

# Read dictionary of lists of selected MSA files for each taxonomic group
with open("results/additional/filtered_msa_dict.pickle", "rb") as handle: 
    selected_MSAs = pickle.load(handle)

# Read dictionary mapping each MSA file to its Gene Symbol
with open("results/additional/SequenceID_to_GeneSymbol.pickle", "rb") as handle: 
    seqID_gene_symbol = pickle.load(handle)

# 2. Search for lysine deserts in OGs

## 2.1. Set parameters

In [2]:
eukaryota = ['4893', '6236', '7214', '9989', '9604']

######################################### LYSINE-LESS REGION ########################################
# Define lysine desert region minimum length; integer or float
desert_min_lengths = [150, 0.5]
# Define minimum fraction of characters in MSA column which are not Lysine
min_desert_fraction = 0.6
# Define minimum fraction of characters in MSA column which are not gaps
min_nongaps_fraction = 0.6
# Define maximum length of continous MSA region for which gap fractions are below min_nongaps_fraction
insertion_length = 15
# Define minimum fraction of MSA sequences which have completely lysine devoid region of given length
min_fraction_seq_lysine_less_region = 0.6 

########################################### LYSINE CLUSTER ##########################################
# Define minimum fraction of total sequence lysines to be in close proximity to form lysine cluster
lysine_cluster_fraction = 0.8
# Define maximum fraction of sequence length to contain defined above fraction of all lysines
lysine_cluster_seq_fraction = 0.2
# Define fraction at the beggining of sequence to be considered as N-terminal
N_term_end_fraction = 0.2
# Define fraction at the end of sequence to be considered as C-terminal
C_term_end_fraction = 0.8
# Minimum percentage of sequences in MSA file to consider MSA as lysine cluster containing
# A less strict threshold for analyzing multiple eukaryotic groups (vectors)
LYSINE_CLUSTER_MSA_THRESHOLD_VECTORS = 0.6 

############################################ CANONICAL AA ###########################################
CANON_AA = ['A', 'G', 'P', 'V', 'L', 'I', 'M', 'C', 'F', 'Y', 
            'W', 'H', 'K', 'R', 'Q', 'N', 'E', 'D', 'S', 'T']

######################################## REFERENCE ORGANISMS ########################################
REFERENCE_ORGANISMS = {'4893': '4932', '6236': '6239', '7214': '7227', '9989': '10090', '9604':'9606'} 

## 2.2. Main code


### Lysine desert search

We define lysine deserts as either:
1. lysine-devoid region of minimum 150 aa length
2. lysine-devoid region of minimum 50% median MSA sequence length

The algorithm distinguishes two types of MSA containing lysine desert region:
* Type **Aligned**, where the lysine desert region of defined length is well conserved in part of the alignment
* Type **Anywhere**, where the lysine desert region of defined length is not conserved in part of the alignment, but most of sequences contain lysine desert region (in any part of the sequence)

The abovementioned lysine desert region classification was introduced due to varying quality of alignments, where some contain very long gaps regions.

The algorithm checks in the first place if MSA has lysine desert region type Aligned, if not checks for lysine desert region type Anywhere.

> **_NOTE:_** Both lysine deserts types (Aligned/Anywhere) are subsequently used to plot phylogenetics trees.

#### Default author's parameters

* Type **Aligned**
  * MSA contains continous region of declared minimum length in which each column has minimum 70% of non-lysines and non-gaps. Small continous insertions (meaning that a few columns in a row contain majority of gaps) are allowed. At least 60% of sequences in that region has to be completely lysine-devoid.
* Type **Anywhere**
  * Each sequence of MSA is checked for presence of completely lysine-less chunk of declared minimum length (it may be anywhere in sequence). At least 80% of sequences have to contain such chunks.




#### Algorithm implementation

The algorithm is as follows:

1. Take MSA, create two masks:

    * Mask 1: if in MSA column at least 60% of characters were not K (lysine) assign 1, 0 otherwise
    * Mask 2: if in MSA column at least 60% of characters were not gaps assign 1, 0 otherwise


2. Merge both masks. 

    * Final Mask = Mask 1 with the exception, that if Mask 2 had fragments with more than declared (default: 15) continous "0" values, then Final Mask will also had "0" values in that fragments. 


3. In the Final Mask, filter fragments with continous "1" values of given length - these will be our lysine desert regions type *Aligned*.

Please note that in type *Aligned*, there may be a small number of Lysines in lysine desert aligned region. However defined fraction of sequences in aligned region (default: at least 60%) will be completely lysine-devoid.


4. If no single lysine desert region type *Aligned* was found, look for type *Anywhere*. 

    In order to do so, take each sequence from MSA and search for lysine-less chunk of given length (in any part of the sequence). If more than defined fraction of sequences possess such chunks, we classify the MSA as type *Anywhere*.


---

### Lysine cluster search

In addition to lysine desert region search, each sequence (**if it contains at least 2 lysines**) is checked for the presence of lysine cluster.

By default lysine cluster is defined as ⌊80% * total number of lysines in sequence⌋ within 20% of sequence length. 

We distinguish three types of lysine cluster, according to its position in sequence:
* anywhere in sequence
* at N-end, defined as first 20% of sequence
* at C-end, defined as last 20% of sequence

For each MSA, fraction of occcurrence of lysine cluster of defined type among all MSA sequences is calculated.

In [3]:
class FindDesertMSA():

    def __init__(self, MSA):
        """Open, read and parse alignment file
        
        :param MSA: path to alignment file
        :type MSA: str
        :return: parsed alignment file object
        :rtype: Bio.Align.MultipleSeqAlignment
        """
        
        f = gzip.open(MSA, "rt")
        alignment = AlignIO.read(f, "fasta")
        f.close()
            
        self.MSA = MSA
        self.og = MSA.split('/')[-2]
        self.msa_id = MSA.split('/')[-1].split('.')[0]
        self.matrix = np.array([list(seq) for seq in alignment], order="F")
        self.names = np.array([alignment[name].id for name in range(len(alignment))])
        self.length = len(self.matrix) # No of sequences in alignment
        
    def _get_seq(self, seq_id, matrix=None):
        """Get sequence without dashes from alignment
        
        :param seq_id: sequence index in alignment matrix
        :type seq_id: int
        :param matrix: matrix
        :type matrix: np.array
        :return: sequence
        :rtype: str
        """
        
        if matrix is None:
            matrix = self.matrix

        return ''.join(matrix[seq_id]).replace('-', '')
        
    def _calculate_desert_fracs(self):
        """Calculate for each column of MSA frequencies of non-K aa (frequency of all aa except K, including gaps)"""
        
        desert_fracs = (np.count_nonzero(self.matrix != 'K', axis = 0) / self.length) 
        return desert_fracs

    def _calculate_nongap_fracs(self):
        """Calculate for each column of MSA frequencies of non-gaps characters."""
        
        nongap_fracs = (np.count_nonzero(self.matrix != '-', axis = 0) / self.length)
        
        return nongap_fracs
    
    def _calculate_MSA_taxons(self):
        """Calculate number of MSA taxons."""
        
        MSA_taxon_set = set()
        for taxon in self.names:
            taxonID = taxon.split('.')[0]
            MSA_taxon_set.add(taxonID)

        return len(MSA_taxon_set)
    
    def calculate_median_seq_length(self, matrix=None):
        """Calculate median of sequence length of a given matrix
        
        :param matrix: matrix
        :type matrix: np.array
        :return: median of sequence length of a given matrix
        :rtype: float
        """
        
        if matrix is None:
            matrix = self.matrix
            
        all_seq_lengths = []
        
        for seq_index in range(len(matrix)):
            all_seq_lengths.append(len(self._get_seq(seq_index, matrix))) 
        
        return statistics.median(all_seq_lengths)
    
    def get_gene_names(self, ref_org=False):
        """Get Gene Symbols for sequences in MSA, or alternatively for reference organism only
        
        :return: gene names list
        :rtype: list
        """
        
        gene_names = set()
        
        for seq_name in self.names:
            if seq_name in seqID_gene_symbol.keys():
                
                if ref_org:
                    if seq_name.split('.')[0] == ref_org:
                        gene_names.add(seqID_gene_symbol[seq_name])
                else:
                    gene_names.add(seqID_gene_symbol[seq_name])
        
        return list(gene_names)
    
    def get_COG(self):
        """Get MSA COG annotation

        :return: COG annotations letter(s)
        :rtype: list
        """

        df_cog = pd.read_csv('data/{}/{}_annotations.tsv.gz'.format(self.og, self.og), sep = '\t', header = None)
        cog = df_cog[df_cog[1] == self.msa_id][2].values
        return ''.join(cog)

    def _check_frac_of_seqs_with_desert(self, matrix):
        """Calculate fraction of sequences in MSA trimmed to lysine desert region which have absolutely no lysine
        
        :param matrix: lysine desert region, trimmed matrix
        :type matrix: np.array
        :return: fraction of sequences which in the given trimmed MSA have abolutely no lysine
        :rtype: list
        """
        
        seq_with_complete_desert = 0
        
        for seq_index in range(len(matrix)):
            seq = self._get_seq(seq_index, matrix)
            K_number = seq.count('K')
            
            if K_number == 0:
                seq_with_complete_desert += 1

        return round(seq_with_complete_desert / self.length, 2)
        
    def _score_desert(self, start, end, desert_nongaps_fracs):
        """Score lysine desert region
        
        :param start: lysine desert region start index in MSA
        :type start: int
        :param end: lysine desert region end index in MSA
        :type end: int
        :param desert_nongaps_fracs: nongap_fracs * desert_fracs
        :type desert_nongaps_fracs: np.array
        :return: average lysine desert conservation, average lysine desert length
        :rtype: list
        """
        
        idx = [i for i in range(start, end)]
        trimmed_matrix = self.matrix[:, idx]
 
        median_desert_length = self.calculate_median_seq_length(trimmed_matrix)
        average_desert_conservation = np.sum(desert_nongaps_fracs[start:end]) / len(trimmed_matrix[0,])
        
        return round(average_desert_conservation, 2), round(median_desert_length, 2)

    
    def _check_MSA_for_potential_desert(self, min_length):
        """If no lysine desert region type Aligned was found, try to find it considering only MSA sequences.
        In other words, check if each sequence of MSA has anywhere in its sequence a defined lysine desert region. 
        If more than declared fraction of sequences (default: 0.8) contain such region, MSA is considerd as lysine desert.
        
        :rtype min_length: int or float
        :return: fraction of sequences that contain lysine desert region of declared length, median of longest lysine desert regions lengths
        :rtype: float, float
        """

        seq_list_no_gaps = []
        
        for seq_index in range(len(self.matrix)):
            seq_list_no_gaps.append(self._get_seq(seq_index, self.matrix)) 
            
        potential_desert_counter = 0
        desert_lengths = []
        
        for seq in range(len(seq_list_no_gaps)):
            longest_desert = 0
            seq_filtered = ''.join(['1' if aa != 'K' else '0' for aa in seq_list_no_gaps[seq]])
            desert_indices_all = [(des.start(), des.end()) for des in re.finditer('1+', seq_filtered)] # get lysine-less sequence chunks

            for indice in desert_indices_all:

                start, end = indice
                length = int(end) - int(start)
                if length >= min_length:
                    if length > longest_desert:
                        longest_desert = length
            
            if longest_desert != 0:
                potential_desert_counter += 1
                desert_lengths.append(longest_desert)

        if potential_desert_counter == 0:
            return 0, 0
        else:
            return round(potential_desert_counter/self.length, 2), statistics.median(desert_lengths)
        
    def calc_median_lysine_less_region_percentage(self):
        """Calculate median percentage of longest lysine-less region per sequence in MSA
        
        :return: median percentage of longest lysine-less region per sequence in MSA
        :rtype: float
        """
        
        lysine_less_lengths = []
        seq_list_no_gaps = []
        
        for seq_index in range(len(self.matrix)):
            seq_list_no_gaps.append(self._get_seq(seq_index, self.matrix)) 
        
        for seq in range(len(seq_list_no_gaps)):
            longest_desert = 0
            seq_filtered = ''.join(['1' if aa != 'K' else '0' for aa in seq_list_no_gaps[seq]])
            desert_indices = [(match.start(), match.end()) for match in re.finditer('1+', seq_filtered)]
            
            for indice in desert_indices:

                start, end = indice
                length = int(end) - int(start)
                if length >= longest_desert:
                    longest_desert = length
            
            lysine_less_lengths.append(round(100*(longest_desert/len(seq_filtered)), 2))

        return round(statistics.median(lysine_less_lengths), 2)
        
    def _check_lysine_cluster(self):
            """Check if lysine cluster region occurs in each MSA sequence. 
            By default, lysine cluster is defined if at least 80% of sequence lysines are within 20% sequence length.
            Minimum number of lysines in sequence is 2 to check it for lysine cluster

            :return: total lysine cluster fraction per sequence in MSA, N-term lysine cluster fraction per sequence in MSA, C-term lysine cluster fraction per sequence in MSA
            :rtype: list
            """
            
            total_cluster_no = 0
            N_cluster_no = 0
            C_cluster_no = 0

            for seq_index in range(len(self.matrix)):
                biggest_cluster = 0
                left_idx = []
                right_idx = []
                seq = self._get_seq(seq_index)
                cluster_max_length = lysine_cluster_seq_fraction * len(seq)

                only_lysine = ''.join(['1' if aa == 'K' else '0' for aa in seq])
                only_lysine_indices = [match.start() for match in re.finditer('1', only_lysine)]

                if len(only_lysine_indices) > 2:
                    # by default 1-lysine_cluster_fraction is 1-0.8=0.2; 80% of lysines have to be present to form lysine cluster
                    for i in range(math.ceil(len(only_lysine_indices)*(1-lysine_cluster_fraction))+1): 
                        for j in range(1,math.ceil(len(only_lysine_indices)*(1-lysine_cluster_fraction))+2-i):
                            if only_lysine_indices[-j] - only_lysine_indices[i] < cluster_max_length:
                                cluster = len(only_lysine_indices[i:-j]) + 1
                                left_idx.append(only_lysine_indices[i])
                                right_idx.append(only_lysine_indices[-j])

                    if len(right_idx) > 0:

                        for right, left in zip(right_idx, left_idx):
                            if right < N_term_end_fraction * len(seq):
                                N_cluster_no += 1
                                break
                            elif left > C_term_end_fraction * len(seq):
                                C_cluster_no += 1
                                break

                        total_cluster_no += 1
                        
                elif len(only_lysine_indices) == 2:
                    if only_lysine_indices[1] - only_lysine_indices[0] < cluster_max_length:
                        total_cluster_no += 1
                        if only_lysine_indices[1] < N_term_end_fraction * len(seq):
                            N_cluster_no += 1
                        if only_lysine_indices[1] > C_term_end_fraction * len(seq):
                            C_cluster_no += 1

            return round(total_cluster_no/len(self.matrix),2), round(N_cluster_no/len(self.matrix),2), round(C_cluster_no/len(self.matrix),2)   
    
    def calculate_avg_aa_per_sequence(self, aa):
        """Calculate average number and average sequence percentage of given amino acid per sequence in MSA 
        
        :param aa: amino acid one letter name 
        :type aa: str
        :return: average number and average sequence percentage of given amino acid per sequence in MSA
        :rtype: float, float
        """
        
        aa_no_in_seq = 0
        percentages = []
        for seq_index in range(len(self.matrix)):
            seq = self._get_seq(seq_index)
            aa_no_in_seq += seq.count(aa.upper())
            percentages.append((seq.count(aa.upper())/len(seq))*100)
            
        return round(aa_no_in_seq/len(self.matrix),2), round(statistics.mean(percentages), 2)
        
    def _check_gene_in_ref_org(self):
        """Checks gene symbol of reference organism

        :return: information gene symbol of reference organism
        :rtype: str
        """
            
        ref_org_genes = set()
        
        for seq_index in range(len(self.matrix)):
            taxon = self.names[seq_index].split('.')[0]
            
            if taxon in REFERENCE_ORGANISMS.values():

                    if self.names[seq_index] in seqID_gene_symbol.keys():
                        ref_org_genes.add(seqID_gene_symbol[self.names[seq_index]])
        
        if len(ref_org_genes) > 0:
            ref_org_genes = list(ref_org_genes)
            ref_org_genes = ';'.join((str(gene) for gene in ref_org_genes))
        else:
            ref_org_genes = None
            
        return ref_org_genes
            
    def _calculate_aa_distributions(self, matrix):
        """For all sequences in given matrix calculate number of each aa occurrence
        
        :param matrix: sequence matrix
        :rtype matrix: np.array
        :return: dictionary with each aa number of occurrence
        :rtype: dict
        """
        
        aa_no = np.zeros(22) # 20 aa + unknown letter + total aa
        aa_dict = {'A' : 0, 'G': 0, 'P': 0, 'V':0, 'L':0, 'I':0, 'M':0, 'C':0, 'F':0, 'Y':0, 'W':0, 'H':0, 'K':0, 'R':0, 'Q':0, 'N':0, 'E':0, 'D':0, 'S':0, 'T':0, 'Other':0, 'Total':0}
        
        for seq_index in range(self.length):
            seq = self._get_seq(seq_index, matrix)
            for residue in seq:
                if residue in CANON_AA: # 20 canonical aa
                    aa_dict[residue] += 1
                else:
                    aa_dict['Other'] += 1
                aa_dict['Total'] += 1
        
        return aa_dict
    
    def find_deserts(self, desert_min_length):
        """Find lysine desert regions of declared length in MSA.
        
        :param desert_min_length: minimum lysine desert region length or sequence fraction
        :type desert_min_length: int or float
        :param min_desert_fraction: minimum fraction of characters in MSA column which are not lysine 
        :type min_desert_fraction: float
        :return: list of lists for each lysine desert region in MSA (may be more than 1), containing information about its properties 
        :rtype: list
        """
        
        gene_names = self.get_gene_names()
        nongap_fracs = self._calculate_nongap_fracs()        
        desert_fracs = self._calculate_desert_fracs()
        
        # 1 if non-gap fraction in col >= min_nongaps_fraction, 0 otherwise
        nongap_fracs_mask = ''.join(['1' if frac >= min_nongaps_fraction else '0' for frac in nongap_fracs]) 
        # 1 if non-lysine fraction in col >= min_desert_fraction, 0 otherwise
        desert_mask = np.array(['1' if frac >= min_desert_fraction else '0' for frac in desert_fracs]) 
        # Find indices of gaps
        gaps_indices = [(chunk.start(), chunk.end()) for chunk in re.finditer('0+', nongap_fracs_mask)] 
        long_gaps_indices = []
        
        for gap in gaps_indices: # save long gaps indices
            gap_length = gap[1] - gap[0]
            if gap_length > insertion_length:
                indices_range = [i for i in range(gap[0], gap[1])]
                long_gaps_indices.extend(indices_range) 
        
        # Filter long gaps from lysine desert regions
        np.put(desert_mask, long_gaps_indices, '0') 
        
        desert_mask = ''.join(desert_mask)
        # Get lysine-less regions
        desert_indices_all = [(des.start(), des.end()) for des in re.finditer('1+', desert_mask)] 
        desert_indices_proper = []
        
        scores = []
        
        if type(desert_min_length) == float:
            desert_min_length = int(desert_min_length * self.calculate_median_seq_length())

        for des_chunk in desert_indices_all:
            indices = [i for i in range(des_chunk[0], des_chunk[1])]
            trimmed_matrix = self.matrix[:, indices]
 
            median_desert_length = self.calculate_median_seq_length(trimmed_matrix)
            
            if median_desert_length >= desert_min_length:
                desert_indices_proper.append((des_chunk[0], des_chunk[1]))
        
        if desert_indices_proper:
            how_many_deserts_in_msa = 0
            for idx in desert_indices_proper:
                indices = [i for i in range(idx[0], idx[1])]
                
                seq_with_lysine_less_region_fraction = self._check_frac_of_seqs_with_desert(self.matrix[:, indices])

                if seq_with_lysine_less_region_fraction >= min_fraction_seq_lysine_less_region:
                    
                    how_many_deserts_in_msa += 1
                    desert_type = 'Aligned'
                    ref_org_genes = self._check_gene_in_ref_org()
                    
                    avg_desert_conservation, median_desert_length =  self._score_desert(idx[0], idx[1], desert_fracs * nongap_fracs)
                    
                    # Calculate median lysine desert length / median protein length in MSA
                    des_length_prot_length = round(median_desert_length/self.calculate_median_seq_length(),2)

                    # Add MSA COG annotations
                    desert_cog = self.get_COG()

                    # Add lysine cluster information
                    total_cluster_no, N_cluster_no, C_cluster_no = self._check_lysine_cluster()
                    
                    # Add avg lysine and arginine number per sequence
                    avg_lys_no_in_seq, avg_lys_percentage_in_seq = self.calculate_avg_aa_per_sequence('K')
                    avg_arg_no_in_seq, avg_arg_percentage_in_seq = self.calculate_avg_aa_per_sequence('R')
                    
                    # Calculate aa occurrence in lysine-less region
                    aa_stats = self._calculate_aa_distributions(self.matrix[:, indices])
                    
                    # Renumber indices according to sequence numeration convention (from 1)
                    start_index = int(idx[0]) + 1
                    if int(idx[1]) == len(self.matrix[0]): # lysine desert stretches to MSA end
                        end_index = int(idx[1])
                    else:
                        end_index = int(idx[1]) - 1
    
                    result = [','.join(gene_names), ref_org_genes, 
                              desert_type, avg_desert_conservation, int(how_many_deserts_in_msa), seq_with_lysine_less_region_fraction, 
                              self.calculate_median_seq_length(), median_desert_length, des_length_prot_length,  
                              start_index , end_index, self.length, round(self.length / self._calculate_MSA_taxons(),2),
                              desert_cog,
                              total_cluster_no, N_cluster_no, C_cluster_no, 
                              avg_lys_no_in_seq, avg_lys_percentage_in_seq,
                              avg_arg_no_in_seq, avg_arg_percentage_in_seq,
                              int(aa_stats['G']), int(aa_stats['P']), int(aa_stats['A']), int(aa_stats['V']), int(aa_stats['L']), int(aa_stats['I']), int(aa_stats['M']),
                              int(aa_stats['C']), int(aa_stats['F']), int(aa_stats['Y']), int(aa_stats['W']), int(aa_stats['H']), int(aa_stats['K']), int(aa_stats['R']),
                              int(aa_stats['Q']), int(aa_stats['N']), int(aa_stats['E']), int(aa_stats['D']), int(aa_stats['S']), int(aa_stats['T']), int(aa_stats['Other']), int(aa_stats['Total'])]
                        
                    scores.append(result)
           
        if len(scores) == 0: # we did not find any Aligned lysine desert region

            check_potential_desert, median_desert_length = self._check_MSA_for_potential_desert(desert_min_length)

            if check_potential_desert >= min_fraction_seq_lysine_less_region:
                
                desert_type = 'Anywhere'
                ref_org_genes = self._check_gene_in_ref_org()
                
                # Calculate median desert length / median protein length in MSA
                des_length_prot_length = round(median_desert_length/self.calculate_median_seq_length(),2)

                # Add MSA COG annotations
                desert_cog = self.get_COG()

                # Add lysine cluster information
                total_cluster_no, N_cluster_no, C_cluster_no = self._check_lysine_cluster()

                # Add avg lysine and arginine number per sequence
                avg_lys_no_in_seq, avg_lys_percentage_in_seq = self.calculate_avg_aa_per_sequence('K')
                avg_arg_no_in_seq, avg_arg_percentage_in_seq = self.calculate_avg_aa_per_sequence('R')
                
                result = [','.join(gene_names), ref_org_genes, 
                          desert_type, None, None, check_potential_desert, self.calculate_median_seq_length(), 
                          median_desert_length, des_length_prot_length, None, None, 
                          self.length, round(self.length / self._calculate_MSA_taxons(), 2),
                          desert_cog,  
                          total_cluster_no, N_cluster_no, C_cluster_no, 
                          avg_lys_no_in_seq, avg_lys_percentage_in_seq,
                          avg_arg_no_in_seq, avg_arg_percentage_in_seq,
                          None, None, None, None, None, None, None, None, None, None, None, 
                          None, None, None, None, None, None, None, None, None, None, None]
                
                scores.append(result)

        return scores

#### Test run

In [4]:
aln = FindDesertMSA('data/4893/4893/3S16B.raw_alg.faa.gz')
print(aln.find_deserts(0.5))

[['san1', 'san1', 'Anywhere', None, None, 0.6, 661, 390, 0.59, None, None, 15, 1.0, 'O', 0.93, 0.0, 0.0, 10.53, 1.6, 39.33, 5.97, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]]


## 2.3. Loop through all selected OGs

Perform lysine desert search on all filtered MSA files for selected eukaryotic families/order.

---

### Summary tables

If function `calculate_lysine_deserts()` is run with argument `save_desert=True`, the summary tables for all found lysine deserts will be saved in tab-separated format (tsv).


#### Naming convention

Summary tables will be saved separately for each taxonomic rank to `results/summary_tables/group_ID`, where group_ID is eggNOG5 ID of analyzed taxonomic rank (identical to NCBI Taxonomy ID).

The naming convention of summary tables is as follows: 

`summary_lysine_desert_150.tsv.gz` or `summary_lysine_desert_0.5.tsv.gz`, 

where 150 or 0.5 refer to analyzed minimum declared lysine desert length.

#### Indexing

Summary tables are indexed by modified MSA ID (relating to original eggNOG5 ID) as follows: 

`MSAId_X` 

where MSAId refers to MSA ID from eggNOG5 database and X indicates occurrence of MSA in summary table, e.g. if MSA contains two different lysine deserts (only in case of **Aligned** lysine desert type), it will occur twice in summary table, indexed as `MSAId_1` and `MSAId_2` for different lysine deserts regions respectively.

#### Columns description

Summary tables contain the following information:

| Column  | Description  |
|:--|:--|
| **All gene symbols**  | All Gene Symbols mapped to MSA  |
|**Reference organism gene symbols** | Gene symbols associated with reference taxon|
| **Lysine desert type**  |  Type of found lysine desert, see Par. 3.1. of `Pipeline.ipynb` |
|  **Avg lysine desert conservation** | Average number of columns with defined fraction of non-gaps characters (default: 70%); provides information about quantity of gaps in lysine desert region.  <br> **Only for Aligned**  lysine desert type  |
| **Lysine deserts no**  | Number of lysine deserts regions.  <br> **Only for Aligned**  lysine desert type   |
|**Fraction of sequences completely lysine devoid** | Fraction of sequences (either within aligned lysine desert region in Aligned type or among entire sequences in Anywhere type) which contain region completely devoid of lysine of given length|
|**Median sequence length** | Median sequence length in MSA|
|**Median lysine desert length** | Median sequence length of lysine desert regions| 
|**Ratio of median lysine desert to median sequence length** | Median lysine desert length divided by median sequence length|
|**Lysine desert start index** | Start index (sequence indexed from 1) of lysine desert region.  <br> **Only for Aligned**  lysine desert type| 
|**Lysine desert end index** | End index (sequence indexed from 1) of lysine desert region.  <br> **Only for Aligned**  lysine desert type| 
|**Number of sequences** | Number of sequences in MSA|
|**Ratio of number of sequences to number of taxons** | Number of sequences divided by number of taxons in MSA|
|**COG** | COG annotation for MSA obtained from eggNOG5 database|
|**Lysine cluster anywhere fraction** | Fraction of sequences in MSA which contain lysine cluster anywhere in their sequences (⌊80% of lysines⌋ located anywhere within 20% of the sequence) |
|**Lysine cluster at N-end fraction** | Fraction of sequences in MSA which contain lysine cluster at their N-ends (⌊80% of lysines⌋ located within the first 20% of sequence)|
|**Lysine cluster at C-end fraction** | Fraction of sequences in MSA which contain lysine cluster at their C-ends (⌊80% of lysines⌋ located within the last 20% of sequence)|
|**Avg no of lysines** | Average number of lysines per sequence in MSA|
|**Avg lysines fraction** | Average fraction of lysines per sequence in MSA|
|**Avg no of arginines** | Average number of arginines per sequence in MSA|
|**Avg arginines fraction** | Average fraction of arginines per sequence in MSA|
|**Glycine no, Proline no, Alanine no, Valine no, Leucine no,  <br> Isoleucine no, Methionine no, Cysteine no, Phenylalanine no,  Tyrosine no, Tryptophan no, Histidine no, Lysine no, Arginine no,  <br> Glutamine no, Asparagine no, Glutamic acid no, Aspartic acid no, Serine no, Threonine no, Other no** | Counted number of given amino acid within lysine desert region.  <br>**Only for Aligned**  lysine desert type| 
|**Total no** | Total number of amino acids in lysine desert region.  <br>**Only for Aligned** lysine desert type| 

In [39]:
def calculate_lysine_deserts(selected_OG, desert_min_length, save_deserts=False):
    """Find lysine-less regions among selected MSA files
    
    :selected_OG: dictionary of taxonomic groups as keys and lists of filtered MSA files as values
    :type selected_OG: dict
    :desert_min_length: minimum lysine-less region length or sequence fraction to consider lysine desert
    :type desert_min_length: int or float
    :save_desert: indicates if summary table with found lysine deserts should be saved
    :type save_desert: bool
    :return: dictionary with list of scores for each taxonomic group
    :rtype: dict
    """
    
    # Dictionary with number of lysine deserts containing MSA files for each taxonomic group
    lysine_desert_total = {}
    
    for group in selected_OG.keys():
        
        group_found_deserts = {}
        desert_counter = 0
        
        for MSA_id in selected_OG[group]:
        
            aln = FindDesertMSA('data/{}/{}/{}.raw_alg.faa.gz'.format(group, group, MSA_id))
            scores = aln.find_deserts(desert_min_length)
            
            if len(scores) > 0:
                desert_counter += 1
            
            for i in range(len(scores)):
                group_found_deserts['{}_{}'.format(MSA_id, i+1)] = scores[i]

        if save_deserts:
            if len(group_found_deserts) > 0:
                
                save_table_path = 'results/summary_tables/{}'.format(group)
                if not os.path.exists(save_table_path):
                    os.makedirs(save_table_path)
                    
                df_csv = pd.DataFrame.from_dict(group_found_deserts, orient='index')
                df_csv.columns = ['All gene symbols', 'Reference organism gene symbols', 
                                  'Lysine desert type', 'Avg lysine desert conservation', 
                                  'Lysine deserts no', 'Fraction of sequences completely lysine devoid ', 
                                  'Median sequence length', 'Median lysine desert length', 
                                  'Ratio of median lysine desert to median sequence length',
                                  'Lysine desert start index', 'Lysine desert end index', 
                                  'Number of sequences', 'Ratio of number of sequences to number of taxons',
                                  'COG',
                                  'Lysine cluster anywhere fraction', 'Lysine cluster at N-end fraction', 
                                  'Lysine cluster at C-end fraction', 
                                  'Avg no of lysines', 'Avg lysines fraction',
                                  'Avg no of arginines', 'Avg arginines fraction',
                                  'Glycine no', 'Proline no', 'Alanine no', 'Valine no', 'Leucine no', 'Isoleucine no', 'Methionine no',
                                  'Cysteine no', 'Phenylalanine no', 'Tyrosine no', 'Tryptophan no', 'Histidine no', 'Lysine no', 'Arginine no',
                                  'Glutamine no', 'Asparagine no', 'Glutamic acid no', 'Aspartic acid no', 'Serine no', 'Threonine no',
                                  'Other no', 'Total no']
                df_csv.index.name = "MSA ID"
                
                df_csv.sort_values(by=['Avg lysine desert conservation'], ascending=False, inplace=True)
                df_csv.to_csv('{}/lysine_desert_{}.tsv.gz'.format(save_table_path, desert_min_length), sep='\t', compression='gzip')
                
        lysine_desert_total[group] = (len(selected_OG[group]), desert_counter)
            
    return lysine_desert_total

## 2.4. Run analysis

In [40]:
eukaryota_dict = {}

for group in selected_MSAs.keys():   
    lineage = ncbi.get_lineage(group)
    if 2759 in lineage:
        eukaryota_dict[group] = selected_MSAs[group]
        
def run_analysis(selected_OG, pickle_out_name, save_deserts=False):
    
    if not os.path.exists('results/trees/pickles'):
        os.makedirs('results/trees/pickles')
    
    for desert_min_length in desert_min_lengths:
        
        result = calculate_lysine_deserts(selected_OG, desert_min_length, save_deserts)
        save_pickle = '{}/{}_lysine_desert_{}.pickle'.format('results/trees/pickles', pickle_out_name, desert_min_length)
            
        with open(save_pickle, 'wb') as handle:
            pickle.dump(result, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [41]:
run_analysis(eukaryota_dict, 'Eukaryota', save_deserts=True)

## 2.5. Plot phylogenetic trees with annotated percentages of lysine desert 

In [20]:
def draw_tree(dictionary, desert_min_length, description):
            
    tree = ncbi.get_topology(dictionary.keys())

    def my_layout(node):

        F = TextFace(node.name, fsize=22, tight_text=True)
        add_face_to_node(F, node, column=0 , position="branch-right")

        if node.is_leaf():
            sciname_face = AttrFace("sci_name", fsize = 28, fgcolor = "black")
            node.add_face(sciname_face, column = 0, position = "branch-right")

            label = dictionary.get(node.name)

            OG = TextFace('    ' + str(label), fsize =26, tight_text = True, bold=True)
            add_face_to_node(OG, node, column = 0 , position = "aligned")
            
        else:
            sciname_face = AttrFace("sci_name", fsize = 24, fgcolor = "black")
            node.add_face(sciname_face, column = 0, position = "branch-right")

    ts = TreeStyle()
    ts.layout_fn = my_layout
    ts.show_leaf_name = False
    ts.optimal_scale_level = "full"
    tree.ladderize()
    
    # Save to file
    tree.render('results/trees/{}_desert_{}.png'.format(description, desert_min_length), tree_style=ts, 
                units='px', w=3000, dpi=600)

Plot phylogenetic trees for selected eukaryotic families/order and annotate them with percentages of MSA files containing lysine desert of defined length in filtered MSA files for a given taxonomic rank. 

> **_NOTE:_**  Only unique MSA lysine desert files are taken into account, meaning that if multiple lysine deserts were found in a single MSA file, it still will be counted as one.

In [21]:
for desert_min_length in desert_min_lengths:
    with open('results/trees/pickles/Eukaryota_lysine_desert_{}.pickle'.format(desert_min_length), 'rb') as handle:
        Eukaryota_results = pickle.load(handle)

    tree_dict = {}
    
    for key in Eukaryota_results.keys():
        # Divide number of MSA with lysine desert by number of filtered MSA files
        tree_dict[key] = str(round(100*(Eukaryota_results[key][1]/Eukaryota_results[key][0]),2))+'%' 
    
    draw_tree(tree_dict, desert_min_length, 'Eukaryota')

# 3. Search for evolutionary conserved lysine deserts in eukaryotic vectors

We will search for lysine deserts in vectors of orthologous MSA (described in par. 5 of `Download_data.ipynb`). 

This approach will allow us to gain evolutionary insight into the process of lysine desert conservation on very different taxonomic levels.

> **_NOTE:_** Before proceeding, run cells in par. 2.1 & 2.2.


## 3.1. Read data

In [23]:
df_vectors = pd.read_csv('results/vectors/eukaryotes_vectors.tsv.gz', sep='\t')

display(Markdown('### Sample of constructed orthologous MSA files table for <br /> Saccharomycetaceae, Rhabditida, Drosophilidae, Rodentia, and Hominidae'))
df_vectors.head()

### Sample of constructed orthologous MSA files table for <br /> Saccharomycetaceae, Rhabditida, Drosophilidae, Rodentia, and Hominidae

,Ophistokonta OG ID,MSA paths,OG summary,Genes,Number of proteins,Number of taxons,Number of taxonomic ranks,Number of MSA files,COG
0,38B3U,"9989/9989/4PSAG.raw_alg.faa.gz,9604/9604/4MZTR...","{'Rodentia': 1, 'Hominidae': 1}","usf3,kiaa2018",179,157,2,2,K
1,38B3V,"6236/6236/40UNT.raw_alg.faa.gz,9989/9989/4PUU6...","{'Rodentia': 4, 'Rhabditida': 1, 'Hominidae': 4}","ctnnd2,arvcf,jac-1,ctnnd1,cbr-jac-1,pkp4",546,152,3,9,TW
2,38B3W,"9604/9604/4N2E5.raw_alg.faa.gz,9989/9989/4Q1E1...","{'Rodentia': 1, 'Hominidae': 1}",fkbp14,142,125,2,2,O
3,38B3X,"9989/9989/4Q2TC.raw_alg.faa.gz,7214/7214/45XNB...","{'Rodentia': 3, 'Rhabditida': 1, 'Drosophilida...","lamc1,lam-2,lanb2,cbr-lam-2,dyak_lanb2,dere_la...",464,163,4,8,W
4,38B3Y,"7214/7214/45VNG.raw_alg.faa.gz,9604/9604/4N26V...","{'Saccharomycetaceae': 1, 'Rhabditida': 1, 'Dr...","nsa2,dyak_ip259,w09c5.1,ip259",387,338,4,4,J


## 3.2. Main code


* We search for lysine deserts in MSA files exactly as before

* We do not distinguish whether MSA has lysine desert type *Aligned* or *Anywhere*; we consider MSA with lysine desert type *Aligned* or *Anywhere* as lysine desert containing MSA.

* We consider MSA as with lysine cluster of given type when:

    * it has lysine desert type *Aligned* or *Anywhere*
    * at least 80% of its sequences have lysine cluster of given type (e.g. for lysine cluster anywhere in the sequence value of `Lysine cluster anywhere fraction` has to be at least 80)
    
* We save the results as tables (see below)

---

## Vectors with calculated lysine deserts 

We will search for lysine desert in each row of the eukaryotic vector table (we will check each MSA file). The results with calculated lysine deserts for MSA files within each vector will be saved as tables.


### Naming convention

Two lysine deserts vector tables will be saved, according to defined minimum length of lysine desert (150 aa or 50% of median MSA sequence). 

The tables will be saved to the following directories: 

* `results/vectors/desert_150/eukaryota_desert_vectors.tsv.gz` 

* `results/vectors/desert_0.5/eukaryota_desert_vectors.tsv.gz` 

where 150 or 0.5 refers to analyzed minimum length of lysine desert.

### Indexing

Summary tables are indexed by modified Ophistokonta Orthologous Group ID.

### Columns description

Summary tables contain the following information:

| Column  | Description  |
|:--|:--|
| **Total taxonomic groups number**  | Total number of selected eukaryotic groups present in the vector  |
| **Total MSA number**  |  Total number of MSA files in the vector |
|  **Ophistokonta COG** | COG annotation for Ophistokonta Orthologous Group (entire vector); note that individual MSA COG annotations may be diffrent from it  |
| ***TAXONOMIC GROUP* total MSA number**  | Total number of MSA files from *TAXONOMIC GROUP* in the vector  |
|***TAXONOMIC GROUP* lysine desert MSAs percentage** | Fraction of lysine desert containing MSA files among all MSA files from given *TAXONOMIC GROUP* in the vector|
|***TAXONOMIC GROUP* non-lysine desert MSAs percentage** | Fraction of non-lysine desert MSA files among all MSA files from given *TAXONOMIC GROUP* in the vector| 
|***TAXONOMIC GROUP* lysine desert gene symbols** | Gene Symbols associated with lysine desert containing MSA files in the vector|
|***TAXONOMIC GROUP* non-lysine desert gene symbols** | Gene Symbols associated with non-lysine desert MSA files in the vector|
|***TAXONOMIC GROUP*  lysine desert MSA COG** | COG annotations of lysine desert MSA files from *TAXONOMIC GROUP* in the vector| 
|***TAXONOMIC GROUP*  non-lysine desert MSA COG** | COG annotations of non-lysine desert MSA files from *TAXONOMIC GROUP* in the vector| 
|***TAXONOMIC GROUP* lysine desert MSAs with lysine cluster anywhere percentage** | Percentage of lysine desert with lysine cluster anywhere in sequence MSA files among all lysine desert MSA files from given *TAXONOMIC GROUP* in the vector|
|***TAXONOMIC GROUP* lysine desert MSAs with lysine cluster N-end percentage** | Percentage of lysine desert with lysine cluster at N-end MSA files among all lysine desert MSA files from given *TAXONOMIC GROUP* in the vector|
|***TAXONOMIC GROUP* lysine desert MSAs with lysine cluster C-end percentage** | Percentage of lysine desert with lysine cluster at C-end MSA files among all lysine desert MSA files from given *TAXONOMIC GROUP* in the vector|
|***TAXONOMIC GROUP* longest lysine-less regions medians among lysine desert MSAs** | Medians of longest lysine less regions in each lysine desert MSA file from given *TAXONOMIC GROUP* in the vector|
|***TAXONOMIC GROUP* longest lysine-less regions medians among non-lysine desert MSAs** | Medians of longest lysine less regions in each non-lysine desert MSA file from given *TAXONOMIC GROUP* in the vector|
|***TAXONOMIC GROUP* means of lysines per sequence in lysine desert MSAs** | Means of number of lysines per sequence in lysine desert MSA file from given *TAXONOMIC GROUP* in the vector |
|***TAXONOMIC GROUP* means of lysines per sequence in non-lysine desert MSAs** |Means of number of lysines per sequence in non-lysine desert MSA file from given *TAXONOMIC GROUP* in the vector |
|***TAXONOMIC GROUP* means of lysines fraction per sequence in lysine desert MSAs** | Means of fraction of lysines per sequence in lysine desert MSA file from given *TAXONOMIC GROUP* in the vector|
|***TAXONOMIC GROUP* means of lysines fraction per sequence in non-lysine desert MSAs** |Means of fraction of lysines per sequence in non-lysine desert MSA file from given *TAXONOMIC GROUP* in the vector |
|***TAXONOMIC GROUP* means of arginines per sequence in lysine desert MSAs** |Means of number of aginines per sequence in lysine desert MSA file from given *TAXONOMIC GROUP* in the vector |
|***TAXONOMIC GROUP* means of arginines per sequence in non-lysine desert MSAs** | Means of number of aginines per sequence in non-lysine desert MSA file from given *TAXONOMIC GROUP* in the vector|
|***TAXONOMIC GROUP* means of arginines fraction per sequence in lysine desert MSAs** | Means of fraction of aginines per sequence in lysine desert MSA file from given *TAXONOMIC GROUP* in the vector|
|***TAXONOMIC GROUP* means of arginines fraction per sequence in non-lysine desert MSAs** |  Means of fraction of aginines per sequence in non-lysine desert MSA file from given *TAXONOMIC GROUP* in the vector|
|***TAXONOMIC GROUP* paths to lysine desert MSAs** | Paths to lysine desert MSA files from given *TAXONOMIC GROUP* in the vector|
|***TAXONOMIC GROUP* paths to non-lysine desert MSAs** | Paths to non-lysine desert MSA files from given *TAXONOMIC GROUP* in the vector|
|***TAXONOMIC GROUP* all MSAs sequence length medians** | Medians of MSA sequence length for all MSA files from given *TAXONOMIC GROUP* in the vector|

*TAXONOMIC GROUP* = *Saccharomycetaceae/Rhabditida/Drosophilidae/Rodentia/Hominidae*

In [24]:
def search_lysine_desert_vectors(desert_min_length, taxonomic_group):
    """Find lysine-less regions by building MSA desert conservation profiles for each aln of OG
    
        :desert_min_lengthh: minimal lysine-less region length or sequence fraction to consider lysine desert
        :type minimal_seq_length: int or float
        :param taxonomic_group: list of NCBI IDs (compatible to created vector file) to search for lysine desert
        :type taxonomic_group: list
        :return: dictionary with lists of lysine desert scores for each taxonomic group within each vector
        :rtype: dict
    """
    
    def vector_division(n, d):
        return round((n / d)*100, 2) if d else None
    
    vectors_lysine_summary = {}

    for ident, value in df_vectors.iterrows():
        
        vectors_lysine_summary[value[0]] = []  
        MSA_list = value[1]
        
        total_MSAs = len(MSA_list.split(','))
        Ophistokonta_COG = value[8]
        
        total_eukaryotic_groups = set()
        total_deserts = 0
        deserts_summary = {}
        
        for group in taxonomic_group:
            deserts_summary[group] = [0,0,0,[],[],[],[],0,0,0,[],[],[],[],[],[],[],[],[],[],[],[],[]] 
            '''no of total MSAs,  
            no of desert MSAs, no of no desert MSAs, 
            MSA desert paths, MSA non-desert paths,
            gene names desert, gene names non desert,
            number of desert MSAs with lysine cluster anywhere, 
            number of desert MSAs with lysine cluster at N-end, 
            number of desert MSAs with lysine cluster at C-end,
            medians of lysine-less region sequence percentages for desert MSAs,
            medians of lysine-less region sequence percentages for non desert MSAs,
            means of number of lysines per sequence in desert MSAs, means of number of lysines per sequence in non-desert MSAs,
            means of fraction of lysines per sequence in desert MSAs, means of fraction of lysines per sequence in non-desert MSAs,
            means of number of arginines per sequence in desert MSAs, means of number of arginines per sequence in non-desert MSAs,
            means of fraction of arginines per sequence in desert MSAs, means of fraction of arginines per sequence in non-desert MSAs,
            COG annotations of desert MSAs, COG annotations of non-desert MSAs, 
            medians of protein length
            '''
        
        for msa in MSA_list.split(','):
            
            taxid = msa.split('/')[0]
            MSA_id = msa.split('/')[-1].split('.')[0]
            total_eukaryotic_groups.add(taxid)
            
            deserts_summary[taxid][0] += 1 # add MSA to total number of MSA files from this taxonomic group

            aln = FindDesertMSA('data/{}/{}/{}.raw_alg.faa.gz'.format(taxid, taxid, MSA_id))
            
            
            # Add median of sequence length
            deserts_summary[taxid][22].append(str(aln.calculate_median_seq_length())) 
            
            # Check if MSA has lysine desert
            scores = aln.find_deserts(desert_min_length)

            # MSA has lysine desert
            if len(scores) > 0:
                deserts_summary[taxid][1] += 1
                deserts_summary[taxid][3].append('data/{}/{}/{}.raw_alg.faa.gz'.format(taxid, taxid, MSA_id))
                deserts_summary[taxid][5].extend(aln.get_gene_names())
                deserts_summary[taxid][10].append(str(aln.calc_median_lysine_less_region_percentage()))

                scores = scores[0]
                # Lysine cluster percentages - how many MSAs in vector of given taxonomic groups have lysine cluster 
                # We classify MSA as with lysine cluster if 80% sequences (default value) have lysine cluster 
                lysine_cluster_anywhere, lysine_cluster_N_end, lysine_cluster_C_end = scores[14], scores[15], scores[16]
                if lysine_cluster_anywhere >= LYSINE_CLUSTER_MSA_THRESHOLD_VECTORS:
                    deserts_summary[taxid][7] += 1
                if lysine_cluster_N_end >= LYSINE_CLUSTER_MSA_THRESHOLD_VECTORS:
                    deserts_summary[taxid][8] += 1
                if lysine_cluster_C_end >= LYSINE_CLUSTER_MSA_THRESHOLD_VECTORS:
                    deserts_summary[taxid][9] += 1
                    
                # Add avg lysine and arginine number per sequence
                lys_no, lys_frac = aln.calculate_avg_aa_per_sequence('K')
                deserts_summary[taxid][12].append(str(lys_no))
                deserts_summary[taxid][14].append(str(lys_frac))
                
                arg_no, arg_frac = aln.calculate_avg_aa_per_sequence('R')
                deserts_summary[taxid][16].append(str(arg_no))
                deserts_summary[taxid][18].append(str(arg_frac))
                
                # Add COG annotation
                deserts_summary[taxid][20].extend(aln.get_COG())
            
            
            else: # MSA does not have lysine desert
                deserts_summary[taxid][2] += 1
                deserts_summary[taxid][4].append('data/{}/{}/{}.raw_alg.faa.gz'.format(taxid, taxid, MSA_id))
                deserts_summary[taxid][6].extend(aln.get_gene_names())
                deserts_summary[taxid][11].append(str(aln.calc_median_lysine_less_region_percentage()))
                
                # Add avg lysine and arginine number per sequence
                lys_no, lys_frac = aln.calculate_avg_aa_per_sequence('K')
                deserts_summary[taxid][13].append(str(lys_no))
                deserts_summary[taxid][15].append(str(lys_frac))
                
                arg_no, arg_frac = aln.calculate_avg_aa_per_sequence('R')
                deserts_summary[taxid][17].append(str(arg_no))
                deserts_summary[taxid][19].append(str(arg_frac))
                
                # Add COG annotation
                deserts_summary[taxid][21].extend(aln.get_COG())
        
        # Save results
        vectors_lysine_summary[value[0]].extend([len(total_eukaryotic_groups), total_MSAs, Ophistokonta_COG])
        
        for group in taxonomic_group:
            # If taxonomic group did not occur in vector
            if deserts_summary[group][0] == 0: 
                vectors_lysine_summary[value[0]].extend([None]*23) # we save 23 columns with different info for each taxonomic group
            
            else:

                lysine_cluster_anywhere_percentage = vector_division(deserts_summary[group][7], deserts_summary[group][1])
                lysine_cluster_N_end_percentage = vector_division(deserts_summary[group][8], deserts_summary[group][1])
                lysine_cluster_C_end_percentage = vector_division(deserts_summary[group][9], deserts_summary[group][1])
                
                vectors_lysine_summary[value[0]].extend([deserts_summary[group][0],
                                                 round((deserts_summary[group][1]/deserts_summary[group][0])*100, 2),
                                                 round((deserts_summary[group][2]/deserts_summary[group][0])*100, 2),
                                                 ';'.join(deserts_summary[group][5]), ';'.join(deserts_summary[group][6]), 
                                                 ';'.join(deserts_summary[group][20]), ';'.join(deserts_summary[group][21]),
                                                 lysine_cluster_anywhere_percentage,
                                                 lysine_cluster_N_end_percentage,
                                                 lysine_cluster_C_end_percentage,
                                                 ';'.join(deserts_summary[group][10]), ';'.join(deserts_summary[group][11]),
                                                 ';'.join(deserts_summary[group][12]), ';'.join(deserts_summary[group][13]),
                                                 ';'.join(deserts_summary[group][14]), ';'.join(deserts_summary[group][15]),
                                                 ';'.join(deserts_summary[group][16]), ';'.join(deserts_summary[group][17]),
                                                 ';'.join(deserts_summary[group][18]), ';'.join(deserts_summary[group][19]),
                                                 ';'.join(deserts_summary[group][3]), ';'.join(deserts_summary[group][4]),
                                                 ';'.join(deserts_summary[group][22])])
                                            
    return vectors_lysine_summary

## 3.3. Run analysis for eukaryotes 

In [25]:
def run_analysis(taxonomic_group, out_name):
    
    for desert_length in desert_min_lengths:
        
        result = search_lysine_desert_vectors(desert_length, eukaryota)
        df = pd.DataFrame.from_dict(result, orient='index')
        df.index.name = 'Ophistokonta OG ID'
        #display(df)
        
        columns = ['Total taxonomic groups number', 'Total MSA number', 'Ophistokonta COG']
        common_col_names = ['total MSA number', 
                            'lysine desert MSAs percentage', 'non-lysine desert MSAs percentage', 
                            'lysine desert gene symbols', 'non-lysine desert gene symbols', 
                            'lysine desert MSA COG', 'non-lysine desert MSA COG',
                            'lysine desert MSAs with lysine cluster anywhere percentage',
                            'lysine desert MSAs with lysine cluster N-end percentage',
                            'lysine desert MSAs with lysine cluster C-end percentage',
                            'longest lysine-less regions medians among lysine desert MSAs',
                            'longest lysine-less regions medians among non-lysine desert MSAs',
                            'means of lysines per sequence in lysine desert MSAs',
                            'means of lysines per sequence in non-lysine desert MSAs',
                            'means of lysines fraction per sequence in lysine desert MSAs',
                            'means of lysines fraction per sequence in non-lysine desert MSAs',
                            'means of arginines per sequence in lysine desert MSAs',
                            'means of arginines per sequence in non-lysine desert MSAs',
                            'means of arginines fraction per sequence in lysine desert MSAs',
                            'means of arginines fraction per sequence in non-lysine desert MSAs',
                            'paths to lysine desert MSAs', 'paths to non-lysine desert MSAs',
                            'all MSAs sequence length medians']
        
        for group in taxonomic_group:
            group_name = ncbi.get_taxid_translator([group])[int(group)]
            for col_name in common_col_names:
                columns.append('{} {}'.format(group_name, col_name))
        
        df.columns = columns
        
        vectors_save_path = 'results/vectors/desert_{}'.format(desert_length)
        if not os.path.exists(vectors_save_path):
            os.makedirs(vectors_save_path)
            
        df.to_csv('results/vectors/desert_{}/{}_desert_vectors.tsv.gz'.format(desert_length, out_name), 
                   sep='\t', compression='gzip')

In [26]:
run_analysis(eukaryota, 'eukaryota')

# 4. Analyze results from vectors 

Analyze results for vectors for **lysine desert of minimum 50% median MSA sequence length**.

### Find highly conserved lysine deserts

Select lysine desert min. 50% conserved in all MSA files from *Drosophilidae, Rodentia*, and *Hominidae* (in *Saccharomycetaceae* and *Rhabditida*, MSAs were either absent or, if present, conserved lysine desert min. 50% also needed to occur).

In [5]:
pd.set_option('display.max_colwidth', None)

df = pd.read_csv('results/vectors/desert_0.5/eukaryota_desert_vectors.tsv.gz', sep='\t')

# all 3 taxonomic groups must be present in table row and all their MSA files have to contain lysine desert
df = df[(df['Drosophilidae lysine desert MSAs percentage'] == 100) & 
        (df['Rodentia lysine desert MSAs percentage'] == 100) &
        (df['Hominidae lysine desert MSAs percentage'] == 100)
       ]

df['Saccharomycetaceae lysine desert MSAs percentage'] = df['Saccharomycetaceae lysine desert MSAs percentage'].replace(np.nan, 101, regex=True)
df['Rhabditida lysine desert MSAs percentage'] = df['Rhabditida lysine desert MSAs percentage'].replace(np.nan, 101, regex=True)
df = df[df['Saccharomycetaceae lysine desert MSAs percentage'] >= 100]
df = df[df['Rhabditida lysine desert MSAs percentage'] >= 100]
df['Saccharomycetaceae lysine desert MSAs percentage'] = df['Saccharomycetaceae lysine desert MSAs percentage'].replace(101, np.nan, regex=True)
df['Rhabditida lysine desert MSAs percentage'] = df['Rhabditida lysine desert MSAs percentage'].replace(101, np.nan, regex=True)

display(df[[
            'Saccharomycetaceae lysine desert gene symbols', 'Saccharomycetaceae paths to lysine desert MSAs',
             'Rhabditida lysine desert gene symbols', 'Rhabditida paths to lysine desert MSAs', 
            'Drosophilidae lysine desert gene symbols', 'Drosophilidae paths to lysine desert MSAs',
             'Rodentia lysine desert gene symbols', 'Rodentia paths to lysine desert MSAs',
             'Hominidae lysine desert gene symbols', 'Hominidae paths to lysine desert MSAs'
            ]])

df.to_csv('results/supplementary/Table_lysine_desert_0.5_conserved_eukaryotic_vectors.tsv', sep='\t')

/var/folders/63/q2_r09gx5q1_k4n6r4pwry5w0000gn/T/ipykernel_19862/664864460.py:3: DtypeWarning: Columns (60,62,64,66,68,83,85,87,89,91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('results/vectors/desert_0.5/eukaryota_desert_vectors.tsv.gz', sep='\t')


,Saccharomycetaceae lysine desert gene symbols,Saccharomycetaceae paths to lysine desert MSAs,Rhabditida lysine desert gene symbols,Rhabditida paths to lysine desert MSAs,Drosophilidae lysine desert gene symbols,Drosophilidae paths to lysine desert MSAs,Rodentia lysine desert gene symbols,Rodentia paths to lysine desert MSAs,Hominidae lysine desert gene symbols,Hominidae paths to lysine desert MSAs
1100,rad23,data/4893/4893/3RYJU.raw_alg.faa.gz,rad-23,data/6236/6236/40YDM.raw_alg.faa.gz,cg10694;rad23,data/7214/7214/45QJE.raw_alg.faa.gz;data/7214/7214/45S3J.raw_alg.faa.gz,rad23a;rad23b,data/9989/9989/4Q4U8.raw_alg.faa.gz;data/9989/9989/4PVU6.raw_alg.faa.gz,rad23b;rad23a,data/9604/9604/4N3HT.raw_alg.faa.gz;data/9604/9604/4MW4P.raw_alg.faa.gz
1149,dsk2,data/4893/4893/3RZAV.raw_alg.faa.gz,ubql-1,data/6236/6236/40XXY.raw_alg.faa.gz,ubqn,data/7214/7214/45N1A.raw_alg.faa.gz,ubqln1;ubqln2,data/9989/9989/4PU7Q.raw_alg.faa.gz;data/9989/9989/4PX5U.raw_alg.faa.gz,ubqln4;ubqln1,data/9604/9604/4MZQ3.raw_alg.faa.gz;data/9604/9604/4N4BU.raw_alg.faa.gz
2135,NaN,NaN,NaN,NaN,ssdp,data/7214/7214/45MWY.raw_alg.faa.gz,ssbp2;ssbp3,data/9989/9989/4Q3QK.raw_alg.faa.gz;data/9989/9989/4Q83H.raw_alg.faa.gz,ssbp3;ssbp2,data/9604/9604/4N31C.raw_alg.faa.gz;data/9604/9604/4N2YF.raw_alg.faa.gz
3262,NaN,NaN,NaN,NaN,NaN,data/7214/7214/45WN3.raw_alg.faa.gz,bag6,data/9989/9989/4Q2H9.raw_alg.faa.gz,bag6,data/9604/9604/4MSAK.raw_alg.faa.gz
3274,NaN,NaN,NaN,NaN,cg11982,data/7214/7214/45MZB.raw_alg.faa.gz,rnf126;rnf115,data/9989/9989/4PW0V.raw_alg.faa.gz;data/9989/9989/4Q1XU.raw_alg.faa.gz,rnf115;rnf126,data/9604/9604/4MS52.raw_alg.faa.gz;data/9604/9604/4N3NE.raw_alg.faa.gz
6582,NaN,NaN,tag-353,data/6236/6236/40S1S.raw_alg.faa.gz,herp,data/7214/7214/45TNG.raw_alg.faa.gz,herpud2;herpud1,data/9989/9989/4PS71.raw_alg.faa.gz;data/9989/9989/4PZKA.raw_alg.faa.gz,herpud2;herpud1,data/9604/9604/4N000.raw_alg.faa.gz;data/9604/9604/4MYAQ.raw_alg.faa.gz
7004,NaN,NaN,NaN,NaN,NaN,data/7214/7214/45QY7.raw_alg.faa.gz,ubl7,data/9989/9989/4PSW9.raw_alg.faa.gz,ubl7,data/9604/9604/4MUZ5.raw_alg.faa.gz
7706,NaN,NaN,toe-4,data/6236/6236/4108P.raw_alg.faa.gz,NaN,data/7214/7214/45SZV.raw_alg.faa.gz,rnf111;rnf165,data/9989/9989/4PZMH.raw_alg.faa.gz;data/9989/9989/4PZMF.raw_alg.faa.gz,rnf165;rnf111,data/9604/9604/4N6CW.raw_alg.faa.gz;data/9604/9604/4MZ0F.raw_alg.faa.gz
10225,mix17,data/4893/4893/3S17Q.raw_alg.faa.gz,har-1,data/6236/6236/411SE.raw_alg.faa.gz,NaN,data/7214/7214/45TPG.raw_alg.faa.gz,chchd2,data/9989/9989/4Q5AB.raw_alg.faa.gz,chchd2,data/9604/9604/4MVN7.raw_alg.faa.gz
